In [1]:
import glob
import math
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
file = glob.glob('/Users/advaitiyer/Desktop/*.csv')[0]
df = pd.read_csv(file, encoding='utf-8', low_memory=False)
df = df[['STATION', 'DATE', 'Station name', 'lat,long', 'Daily nowDep h', 'Daily nowfall', 'HourlyDryBulbtemperature', 'HourlyPrecipitation', 'Hourly visibility', 'HourlyWindspeed']]
df['DATE'] = pd.to_datetime(df['DATE'])
df['DATE']=df['DATE'].dt.round('H')
df.columns = ['station_id', 'datetime', 'station_name', 'latlong', 'snow_depth', 'snowfall', 'drybulb_temp', 'precipitation', 'visibility', 'windspeed']
df['date'] = df['datetime'].dt.date
df

,station_id,datetime,station_name,latlong,snow_depth,snowfall,drybulb_temp,precipitation,visibility,windspeed,date
0,72505394728,2016-01-01 01:00:00,"NY CITY CENTRAL PARK, NY U","40.77898, -73.96925",NaN,NaN,42.0,0,10.0,5.0,2016-01-01
1,72505394728,2016-01-01 02:00:00,"NY CITY CENTRAL PARK, NY U","40.77898, -73.96925",NaN,NaN,41.0,0,10.0,3.0,2016-01-01
2,72505394728,2016-01-01 03:00:00,"NY CITY CENTRAL PARK, NY U","40.77898, -73.96925",NaN,NaN,41.0,0,10.0,5.0,2016-01-01
3,72505394728,2016-01-01 04:00:00,"NY CITY CENTRAL PARK, NY U","40.77898, -73.96925",NaN,NaN,41.0,0,10.0,9.0,2016-01-01
4,72505394728,2016-01-01 05:00:00,"NY CITY CENTRAL PARK, NY U","40.77898, -73.96925",NaN,NaN,40.0,0,10.0,10.0,2016-01-01
...,...,...,...,...,...,...,...,...,...,...,...
23320,72502014734,2016-06-30 22:00:00,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ U","40.6825, -74.1694",NaN,NaN,76.0,0,10.0,10.0,2016-06-30
23321,72502014734,2016-06-30 23:00:00,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ U","40.6825, -74.1694",NaN,NaN,76.0,0,10.0,10.0,2016-06-30
23322,72502014734,2016-07-01 00:00:00,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ U","40.6825, -74.1694",NaN,NaN,74.0,0,10.0,8.0,2016-07-01
23323,72502014734,2016-07-01 00:00:00,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ U","40.6825, -74.1694",0,0,NaN,NaN,NaN,NaN,2016-07-01


In [3]:
def process1(df):
    ls = []
    for i in range(len(df)):
        if [df['snow_depth'][i],df['snowfall'][i]] != [np.nan, np.nan]:
            ls.append((df['date'][i],df['station_id'][i], (df['snow_depth'][i],df['snowfall'][i])))
    return ls

In [4]:
a = process1(df)

In [5]:
def process2(ls):
    d = {x: [] for x, _, _ in ls}
    for i, j, k in ls:
        d[i] += [(j,k[0],k[1])]
        df = pd.DataFrame(d.items(), columns = ['date', 'attributes'])
        df = df.explode('attributes')
        df[['c1','c2','c3']] = pd.DataFrame(df['attributes'].tolist(), index=df.index)
        df = df[['date','c1','c2','c3']]
        df.columns = ['date','station_id','snow_depth','snowfall']
    return df

In [6]:
d = process2(a)

In [7]:
out = pd.merge(df, d, how='left', left_on=['date','station_id'], right_on=['date','station_id'])
out = out[['station_id','datetime','station_name','latlong','drybulb_temp','precipitation','visibility','windspeed','date','snow_depth_y','snowfall_y']]
# out.to_csv(r'/Users/advaitiyer/Desktop/output.csv', index=True, sep='\t')

In [8]:
groups = out.groupby(['datetime', 'station_id'])
print("Station ID NAs:", out.station_id.isnull().sum() * 100 / len(df), "%")
print("Datetime NAs:", out.datetime.isnull().sum() * 100 / len(df), "%")
print("Station-name NAs:", out.station_name.isnull().sum() * 100 / len(df), "%")
print("Lat-long NAs:", out.latlong.isnull().sum() * 100 / len(df), "%")
print("Visibility NAs:", out.snowfall_y.isnull().sum() * 100 / len(df), "%")
print("Drybulb NAs:", out.drybulb_temp.isnull().sum() * 100 / len(df), "%")
print("Precipitation NAs:", out.precipitation.isnull().sum() * 100 / len(df), "%")
print("Visibility NAs:", out.snowfall_y.isnull().sum() * 100 / len(df), "%")
print("Windspeed NAs:", out.windspeed.isnull().sum() * 100 / len(df), "%")
print("Snow-depth NAs:", out.snow_depth_y.isnull().sum() * 100 / len(df), "%")
print("Snowfall NAs:", out.snowfall_y.isnull().sum() * 100 / len(df), "%")

Station ID NAs: 0.0 %
Datetime NAs: 0.0 %
Station-name NAs: 0.0 %
Lat-long NAs: 0.0 %
Visibility NAs: 22.70096463022508 %
Drybulb NAs: 3.2282958199356915 %
Precipitation NAs: 19.584137191854232 %
Visibility NAs: 22.70096463022508 %
Windspeed NAs: 4.831725616291533 %
Snow-depth NAs: 23.031082529474812 %
Snowfall NAs: 22.70096463022508 %


In [121]:
for name, group in groups:
    inter_list = []
    for elements in group.iterrows():
        inter_list.append(elements)
#         break
    break
inter_list

[(17464, station_id                                      72502014734
  datetime                                2016-01-01 01:00:00
  station_name     NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ U
  latlong                                   40.6825, -74.1694
  drybulb_temp                                             42
  precipitation                                             0
  visibility                                               10
  windspeed                                                 8
  date                                             2016-01-01
  snow_depth_y                                            NaN
  snowfall_y                                              NaN
  Name: 17464, dtype: object),
 (17465, station_id                                      72502014734
  datetime                                2016-01-01 01:00:00
  station_name     NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ U
  latlong                                   40.6825, -74.1694
  drybulb_temp           

In [133]:
def interpolator(groups, val):
    master_list = []
    for name, group in groups:
        lis = []
        for elements in group.iterrows():
            lis.append(elements)
        inter_list = []
        for i in range(len(lis)):
            try:
                inter_list.append(float(lis[i][1][val]))
            except:
                print("ValueError: could not convert string to float:")
            try:
                percentage = (inter_list.count(np.nan) / len(inter_list) * 100)
            except:
                print("ZeroDivisionError: division by zero")
            if percentage == 100.0:
                for i in range(len(inter_list)):
                    inter_list[i] = 0.0
            elif percentage > 0.0:
                for i in range(len(inter_list)):
                    inter_list[i] = max(inter_list)
            elif percentage == 0.0:
                j = np.mean(inter_list)
                for i in range(len(inter_list)):
                    inter_list[i] = j
        master_list.append(inter_list)
    return master_list

In [134]:
interpolator(groups, 'drybulb_temp')

[[42.0, 42.0],
 [42.0],
 [43.0, 43.0],
 [42.0],
 [42.0],
 [41.0],
 [43.0],
 [41.0],
 [41.0],
 [41.0],
 [42.0],
 [41.0],
 [41.0],
 [41.0],
 [42.0, 42.0],
 [41.0],
 [40.0],
 [40.0],
 [41.0],
 [40.0],
 [40.0],
 [40.0],
 [41.0],
 [40.0],
 [39.0, 39.0],
 [39.0],
 [41.0, 41.0],
 [39.0],
 [39.0],
 [39.0],
 [41.0],
 [39.0],
 [39.0],
 [40.0],
 [40.0],
 [39.0],
 [40.0],
 [40.0],
 [40.0, 40.0],
 [40.0],
 [41.0],
 [41.0],
 [41.0],
 [40.0],
 [41.0],
 [41.0],
 [42.0],
 [40.0],
 [41.0, 41.0],
 [41.0],
 [41.0, 41.0],
 [40.0],
 [40.0],
 [40.0],
 [41.0],
 [40.0],
 [39.0],
 [39.0],
 [40.0],
 [39.0],
 [39.0],
 [39.0],
 [40.0, 40.0],
 [39.0],
 [38.0],
 [38.0],
 [40.0],
 [38.0],
 [38.0],
 [38.0],
 [39.0],
 [38.0],
 [38.0, 38.0],
 [37.0],
 [39.0, 39.0],
 [37.0],
 [37.0],
 [36.0],
 [38.0],
 [37.0],
 [36.0],
 [35.0],
 [38.0],
 [36.0],
 [35.0],
 [35.0],
 [37.0, 37.0],
 [35.0],
 [34.0],
 [34.0],
 [36.0],
 [35.0],
 [nan, nan],
 [nan, nan],
 [nan, nan],
 [nan, nan],
 [33.0, 33.0],
 [34.0],
 [36.0, 36.0],
 [34.0],


In [135]:
interpolator(groups, 'precipitation')

ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero


/Users/advaitiyer/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/advaitiyer/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
Valu

ValueError: could not convert string to float:
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ValueError: could not convert string to float:
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError:

ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ValueError: could not convert string to float:
ValueError: could not convert string to float:
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert

...

ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ValueError: could not convert string to float:
ValueError: could not convert string to float:
ValueError: could not convert string to float:
ValueError: could not convert string to float:
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could 

ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ValueError: could not convert string to float:
ValueError: could not convert string to float:
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ValueError: could not convert string to float:
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could 

ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
Valu

ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
Valu

[[0.0, 0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 

In [136]:
interpolator(groups, 'visibility')

[[9.969999999999999, 9.969999999999999],
 [10.0],
 [9.969999999999999, 9.969999999999999],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [9.969999999999999, 9.969999999999999],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [9.969999999999999, 9.969999999999999],
 [10.0],
 [9.969999999999999, 9.969999999999999],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [9.969999999999999, 9.969999999999999],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [9.969999999999999, 9.969999999999999],
 [10.0],
 [9.969999999999999, 9.969999999999999],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [9.969999999999999, 9.969999999999999],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [10.0],
 [9.969999999999999, 9.969999999999999],
 [10.0],
 [9.9699999999

In [137]:
interpolator(groups, 'snow_depth_y')

ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
Valu

ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
Zero

ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
Valu

ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
Valu

[[0.0, 0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 

In [138]:
interpolator(groups, 'snowfall_y')

ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
Valu

ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
Valu

ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
Valu

ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
Valu

ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
Valu

ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
Valu

ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
Valu

ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
Valu

ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
ValueError: could not convert string to float:
ZeroDivisionError: division by zero
Valu

[[0.0, 0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0, 0.0],
 [0.0],
 [0.0],
 

In [76]:
drybulb_temp = []
precipitation = []
visibility = []
windspeed = []
date = []
snow_depth_y = [] 
snowfall_y = []
for i in range(len(inter_list)):
    drybulb_temp.append(float(inter_list[i][1]['drybulb_temp']))
    for n, i in enumerate(drybulb_temp):
        if (str(i) == 'nan' or str(i) == " " or str(i) == ""):
            drybulb_temp[n] = 
    precipitation.append(float(inter_list[i][1]['precipitation']))
    visibility.append(float(inter_list[i][1]['visibility']))
    windspeed.append(float(inter_list[i][1]['windspeed']))
    snow_depth_y.append(float(inter_list[i][1]['snow_depth_y']))
    snowfall_y.append(float(inter_list[i][1]['snowfall_y']))
print(drybulb_temp)
print(precipitation)
print(visibility)
print(windspeed)
print(snow_depth_y)
print(snowfall_y)               
                

[42.0, 42.0]
[0.0, nan]
[10.0, 9.94]
[8.0, 8.0]
[nan, nan]
[nan, nan]


In [ ]:
for name, group in groups:
    precip=[]
    clear=[]
    for index, row in group.iterrows():
        pc = row['precipitation']
        incoms=data['int_income'].dropna().unique().tolist()
        pc=float(pc)
        pc=pc.dropna()
        print(pc)
        pc =str(pc)
        
#         precip.append(pc)
        if (pc == ' 'or pc=='nan'):
            val="x"
            precip.append(pc)
        else:
            precip.append(pc)
    for i in precip:
        if i != "x":
            i=float(i)
            clear.append(i)
    break
    print(clear)
        
        
    

    
            nul.append(val)
        else:
            precip.append(val)
        break
    print (nul)
    print(precip)
        if pc != 'nan' or pc!=" " or pc != "" or p!=nan:
            precip.append(pc)
    print (precip)


In [ ]:
print(precip)